损失函数

In [ ]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
import mindspore.dataset as ds
import mindspore as ms

loss = nn.L1Loss()
output_data = Tensor(np.array([[1, 2, 3], [2, 3, 4]]).astype(np.float32))
target_data = Tensor(np.array([[0, 2, 5], [3, 1, 1]]).astype(np.float32))
print(loss(output_data, target_data))

优化器

In [ ]:
optim = nn.Momentum(params=modelle.trainable_params(), learning_rate=0.1, momentum=0.9, weight_decay=0.0)

模型编译

In [ ]:
from mindspore import Model

# 定义神经网络
net = LeNet5()
# 定义损失函数
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
# 定义优化器
optim = nn.Momentum(params=net.trainable_params(), learning_rate=0.1, momentum=0.9)
# 模型编译
model = Model(net, loss_fn=loss, optimizer=optim, metrics={'accuracy'})

模型训练

In [ ]:
import mindspore.dataset.transforms.c_transforms as C
import mindspore.dataset.vision.c_transforms as CV
from mindspore.train.callback import LossMonitor

DATA_DIR = "./MNIST_Data/train"
mnist_dataset = ds.MnistDataset(DATA_DIR)

resize_op = CV.Resize((28, 28))
rescale_op = CV.Rescale(1 / 255, 0)
hwc2chw_op = CV.HWC2CHW()

mnist_dataset = mnist_dataset.map(input_columns="image", operations=[rescale_op, resize_op, hwc2chw_op])
mnist_dataset = mnist_dataset.map(input_columns="label", operations=C.TypeCast(mstype.int32))
mnist_dataset = mnist_dataset.batch(32)
loss_cb = LossMonitor(per_print_times=1000)
# dataset 是传入参数代表训练集，epoch是训练集迭代次数
model.train(epoch=1, train_dataset=mnist_dataset, callbacks=[loss_cb])

模型评估

In [ ]:
# 测试集
DATA_DIR = "./MNIST_Data/test"
dataset = ds.MnistDataset(DATA_DIR)

resize_op = CV.Resize((28, 28))
rescale_op = CV.Rescale(1 / 255, 0)
hwc2chw_op = CV.HWC2CHW()

dataset = dataset.map(input_columns="image", operations=[rescale_op, resize_op, hwc2chw_op])
dataset = dataset.map(input_columns="label", operations=C.TypeCast(mstype.int32))
dataset = dataset.batch(32)
model.eval(val_dataset=dataset)